In [1]:
# 1 get accelerometer and gyroscope and operation labels
# 2 resample the data according to size of video frames

# https://open-pack.github.io/gallery/modality/
# atr01:right hand, atr02:left hand

# 1 get accelerometer and gyroscope and operation labels

In [1]:
import os
import csv
import numpy as np
import pandas as pd
from datetime import datetime
# 



In [2]:
os.getcwd()

'D:\\code\\DeepConvLSTM-Pytorch\\data\\raw\\openpack'

In [4]:
# read raw sensor(acc,gyro) data and label
user = 'U0101'
scenarios = ['S0100', 'S0200','S0300','S0400','S0500']
labellist, rimulist, limulist = [],[],[]
for scenario in scenarios:
    label_path = r'%s\annotation\activity-1s\%s.csv' % (user, scenario)
    # label_path = r'%s\annotation\openpack-operations\%s.csv' % (user, scenario)
    tfull_label = pd.read_csv(label_path)
    
    rIMU_path = r'%s\atr\atr01\%s.csv' % (user, scenario)
    trIMU = pd.read_csv(rIMU_path)
    
    lIMU_path = r'%s\atr\atr02\%s.csv' % (user, scenario)
    tlIMU = pd.read_csv(lIMU_path)
    labellist.append(tfull_label)
    rimulist.append(trIMU)
    limulist.append(tlIMU)
full_label = pd.concat(labellist)
rIMU = pd.concat(rimulist)
lIMU = pd.concat(limulist)


In [5]:
len(full_label)

9108

In [6]:
len(rIMU)

276607

In [45]:
# convert start/end time in label to unix
# timestamp_str = '2022-03-25 09:58:21.450000+09:00'
# datetime_obj = datetime.fromisoformat(timestamp_str)
# unix_timestamp = int(datetime_obj.timestamp() * 1000)
full_label['startunix'] = full_label['start'].map(lambda x: int(datetime.fromisoformat(x).timestamp()*1000))
full_label['endunix'] = full_label['end'].map(lambda x: int(datetime.fromisoformat(x).timestamp()*1000))
full_label.head(2)

,uuid,user,session,box,id,operation,start,end,actions,startunix,endunix
0,op:f59ab8a0-6d65-5ec3-b2af-eae1a8f00d6c,U0210,S0100,1,100,Picking,2022-04-01 09:38:13.146000+09:00,2022-04-01 09:38:23.745000+09:00,act:a68c9b32-6b1a-52b6-bbf6-5adaf3b55377;act:4...,1648773493146,1648773503745
1,op:6e35aef3-aae8-5b5f-a316-62820def97cb,U0210,S0100,1,200,Relocate Item Label,2022-04-01 09:38:23.745000+09:00,2022-04-01 09:38:30.301000+09:00,act:e511c4a9-a592-53b4-bd44-20a6b650cfc7;act:f...,1648773503745,1648773510301


In [46]:
# Merging logic
def get_label(row):
    # Find the label for the timestamp if it falls within any range in b
    mask = (full_label['startunix'] <= row['unixtime']) & (full_label['endunix'] >= row['unixtime'])
    if any(mask):
        return full_label.loc[mask, 'operation'].values[0]
    return None

In [47]:
rIMU['label'] = rIMU.apply(get_label, axis=1)

In [48]:
lIMU['label'] = lIMU.apply(get_label, axis=1)

In [49]:
# combin right left imu

allIMU = pd.merge(lIMU[['unixtime', 'acc_x','acc_y', 'acc_z', 'gyro_x','gyro_y', 'gyro_z']], 
                  rIMU[['unixtime','acc_x','acc_y', 'acc_z', 'gyro_x','gyro_y', 'gyro_z', 'label']], 
                  how='inner', on='unixtime')

In [50]:
allIMU.head(2)

,unixtime,acc_x_x,acc_y_x,acc_z_x,gyro_x_x,gyro_y_x,gyro_z_x,acc_x_y,acc_y_y,acc_z_y,gyro_x_y,gyro_y_y,gyro_z_y,label
0,1648773491010,0.2860,-0.9614,0.0677,0.48,2.19,-1.15,0.2462,0.9674,0.0539,0.24,1.89,-0.06,None
1,1648773491040,0.2866,-0.9605,0.0683,0.36,1.89,-0.97,0.2403,0.9699,0.0540,0.12,2.01,-0.18,None


In [51]:
output_path = r'left_right_IMU_%s_%s.npy'%(user, scenario)
with open(output_path, 'wb') as f:
    np.save(f, allIMU)